In [3]:
import torch
import numpy as np
import torchvision.transforms as transforms
#from myRNN import RNN
from time import sleep
import sys
#from expectimax import board_to_move
#from game import Game
#from displays import Display
import csv, os
#PATH = '/home/lqgao/2048-api/model/rnn_model_21.pkl'
PATH = 'rnn_model_15.pkl'
PATH2 = '/home/fengdahu/2048-api/model/rnn_model_20final.pkl'
PATH1 = '/home/fengdahu/2048-api/model/myRNN3Random05Model50rate:0.pkl'
dataSetFilename = 'Train.csv'

In [2]:
class Agent:
    '''Agent Base.'''

    def __init__(self, game, display=None):
        self.game = game
        self.display = display

    def play(self, max_iter=np.inf, verbose=False):
        n_iter = 0
        while (n_iter < max_iter) and (not self.game.end):
            direction = self.step()
            self.game.move(direction)
            n_iter += 1
            if verbose:
                print("Iter: {}".format(n_iter))
                print("======Direction: {}======".format(
                    ["left", "down", "right", "up"][direction]))
                if self.display is not None:
                    self.display.display(self.game)

    def step(self):
        direction = int(input("0: left, 1: down, 2: right, 3: up = ")) % 4
        return direction

In [4]:
class MyAgent(Agent):

    def __init__(self, game, display=None):
        if game.size != 4:
            raise ValueError(
                "`%s` can only work with game of `size` 4." % self.__class__.__name__)
        super().__init__(game, display)
        # from .expectimax import board_to_move
        # self.search_func = board_to_move
        # self.search_func = np.random.randint(0, 4)

        self.model = torch.load(PATH)
        self.model.eval()

    def step(self):

        # tmp = self.game.board.flatten()
        # print(board)
        board = np.where(self.game.board == 0, 1, self.game.board)
        board = np.log2(board)
        # print(board)
        board = board.reshape((4, 4))
        # sleep(3600)
        board = board[:, :, np.newaxis]
        board = board / 11.0
        trans = transforms.Compose([transforms.ToTensor()])
        board = trans(board)
        board = torch.unsqueeze(board, dim=0)
        board = board.type(torch.float)
        out = self.model(board)
        direction = torch.max(out, 1)[1]
        return int(direction)